<a href="https://colab.research.google.com/github/reidhulsizer/Profile/blob/master/NFLspike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
files.upload()

Saving BBMIV Player Combos - All Players.csv to BBMIV Player Combos - All Players.csv


{'BBMIV Player Combos - All Players.csv': b"Player,Team,Position,Drafted %,Justin Jefferson,Ja'Marr Chase,Christian McCaffrey,Tyreek Hill,Cooper Kupp,Travis Kelce,Stefon Diggs,A.J. Brown,Bijan Robinson,Austin Ekeler,CeeDee Lamb,Davante Adams,Jonathan Taylor,Garrett Wilson,Amon-Ra St. Brown,Saquon Barkley,Jaylen Waddle,Patrick Mahomes,Nick Chubb,Jalen Hurts,Chris Olave,Josh Allen,Tony Pollard,DeVonta Smith,Tee Higgins,Derrick Henry,Josh Jacobs,Breece Hall,Rhamondre Stevenson,DK Metcalf,Mark Andrews,Deebo Samuel,Calvin Ridley,Lamar Jackson,Amari Cooper,Travis Etienne,Justin Fields,Najee Harris,Jahmyr Gibbs,DeAndre Hopkins,Christian Watson,Joe Burrow,Mike Williams,Jerry Jeudy,DJ Moore,Keenan Allen,Kenneth Walker,T.J. Hockenson,Drake London,Terry McLaurin,Christian Kirk,Justin Herbert,George Kittle,Aaron Jones,Michael Pittman,J.K. Dobbins,Jaxon Smith-Njigba,Miles Sanders,Chris Godwin,Brandon Aiyuk,Trevor Lawrence,Kyle Pitts,Joe Mixon,Dameon Pierce,Tyler Lockett,Mike Evans,Dallas Goedert,Ma

In [3]:
import pandas as pd
import os

In [12]:
df = pd.read_csv('BBMIV Player Combos - All Players.csv')
def replace_percent_and_convert_to_int(x):
    return int(x.replace('%', ''))

# Apply the function to all columns in the subset
df.iloc[:, 3:] = df.iloc[:, 3:].applymap(replace_percent_and_convert_to_int)
players = df.iloc[:,:4]
teams = pd.DataFrame(pd.unique(players.Team))
teams.columns = ['team']
teams = teams.iloc[:32,:]
teams['opponent'] = ''
teams['opponent'][0] = teams['team'][25]
teams['opponent'][1] = teams['team'][5]
teams['opponent'][2] = teams['team'][27]
teams['opponent'][3] = teams['team'][21]
teams['opponent'][4] = teams['team'][15]
teams['opponent'][5] = teams['team'][1]
teams['opponent'][6] = teams['team'][19]
teams['opponent'][7] = teams['team'][31]
teams['opponent'][8] = teams['team'][23]
teams['opponent'][9] = teams['team'][26]
teams['opponent'][10] = teams['team'][14]
teams['opponent'][11] = teams['team'][12]
teams['opponent'][12] = teams['team'][11]
teams['opponent'][13] = teams['team'][16]
teams['opponent'][14] = teams['team'][10]
teams['opponent'][15] = teams['team'][4]
teams['opponent'][16] = teams['team'][13]
teams['opponent'][17] = teams['team'][29]
teams['opponent'][18] = teams['team'][30]
teams['opponent'][19] = teams['team'][6]
teams['opponent'][20] = teams['team'][24]
teams['opponent'][21] = teams['team'][3]
teams['opponent'][22] = teams['team'][28]
teams['opponent'][23] = teams['team'][8]
teams['opponent'][24] = teams['team'][20]
teams['opponent'][25] = teams['team'][0]
teams['opponent'][26] = teams['team'][9]
teams['opponent'][27] = teams['team'][2]
teams['opponent'][28] = teams['team'][22]
teams['opponent'][29] = teams['team'][17]
teams['opponent'][30] = teams['team'][18]
teams['opponent'][31] = teams['team'][7]
teams.columns = ['Team', 'Opponent']
players = pd.merge(players, teams, on='Team')
df['Rank'] = df.index
df = pd.merge(df,teams,on='Team')

In [17]:
# Step 1: Filter the DataFrame to keep only players with 'Position' == 'WR' or 'TE'
filtered_df = df[(df['Position'] == 'WR') | (df['Position'] == 'TE')].copy()

# Step 2: Create a DataFrame containing only QBs and their ranks
qbs_df = df[df['Position'] == 'QB'][['Team', 'Player', 'Rank']]

# Step 3: Find the lowest-ranked QB for each team
lowest_rank_qb_df = qbs_df.loc[qbs_df.groupby('Team')['Rank'].idxmin()]

# Step 4: Merge the filtered DataFrame with the DataFrame containing the lowest-ranked QBs for the same team
merged_df_same_team = pd.merge(filtered_df, lowest_rank_qb_df, left_on='Team', right_on='Team', suffixes=('_WR/TE', '_QB_SameTeam'))

# Step 5: Replace the same team quarterback name with the corresponding value from the df DataFrame
def get_same_team_qb_value(row):
    qb_name = row['Player_QB_SameTeam']
    player_name = row['Player_WR/TE']
    qb_value = df.loc[df['Player'] == player_name, qb_name].values[0]
    return qb_value

merged_df_same_team['Lowest_Rank_QB_SameTeam'] = merged_df_same_team.apply(get_same_team_qb_value, axis=1)

# Step 6: Merge the filtered DataFrame with the DataFrame containing the lowest-ranked QBs for the opponent team
merged_df_opponent_team = pd.merge(filtered_df, lowest_rank_qb_df, left_on='Opponent', right_on='Team', suffixes=('_WR/TE', '_QB_Opponent'))

# Step 7: Replace the opponent team quarterback name with the corresponding value from the df DataFrame
def get_opponent_team_qb_value(row):
    qb_name = row['Player_QB_Opponent']
    player_name = row['Player_WR/TE']
    qb_value = df.loc[df['Player'] == player_name, qb_name].values[0]
    return qb_value

merged_df_opponent_team['Lowest_Rank_QB_Opponent'] = merged_df_opponent_team.apply(get_opponent_team_qb_value, axis=1)

# Step 8: Create a DataFrame containing only WRs and their ranks
wr_df = df[df['Position'] == 'WR'][['Team', 'Player', 'Rank']]

# Step 9: Find the lowest-ranked WR for each team
lowest_rank_wr_df = wr_df.loc[wr_df.groupby('Team')['Rank'].idxmin()]

# Step 10: Find the second lowest-ranked WR for each team
second_lowest_rank_wr_df = wr_df.loc[wr_df.groupby('Team')['Rank'].apply(lambda x: x.nsmallest(2).idxmax())]

# Step 11: Find the third lowest-ranked WR for each team
third_lowest_rank_wr_df = wr_df.loc[wr_df.groupby('Team')['Rank'].apply(lambda x: x.nsmallest(3).idxmax())]

# Step 12: Create a DataFrame containing only TEs and their ranks
tes_df = df[df['Position'] == 'TE'][['Team', 'Player', 'Rank']]

# Step 13: Find the lowest-ranked TE for each team
lowest_rank_te_df = tes_df.loc[tes_df.groupby('Team')['Rank'].idxmin()]

# Step 14: Merge the filtered DataFrame with the DataFrame containing the lowest-ranked TEs for the opponent team
merged_df_opponent_te = pd.merge(filtered_df, lowest_rank_te_df, left_on='Opponent', right_on='Team', suffixes=('_WR/TE', '_TE_Opponent'))

# Step 15: Replace the opponent team TE name with the corresponding value from the df DataFrame
def get_opponent_team_te_value(row):
    te_name = row['Player_TE_Opponent']
    player_name = row['Player_WR/TE']
    te_value = df.loc[df['Player'] == player_name, te_name].values[0]
    return te_value

merged_df_opponent_te['Lowest_Rank_TE_Opponent'] = merged_df_opponent_te.apply(get_opponent_team_te_value, axis=1)

# Step 9: Find the lowest-ranked WR for each team
lowest_rank_wr_df = wr_df.loc[wr_df.groupby('Team')['Rank'].idxmin()]

# Step 10: Find the second lowest-ranked WR for each team
second_lowest_rank_wr_df = wr_df.loc[wr_df.groupby('Team')['Rank'].apply(lambda x: x.nsmallest(2).idxmax())]

# Step 11: Find the third lowest-ranked WR for each team
third_lowest_rank_wr_df = wr_df.loc[wr_df.groupby('Team')['Rank'].apply(lambda x: x.nsmallest(3).idxmax())]

# Step 12: Merge the filtered WR DataFrame with the DataFrame containing the lowest-ranked WRs for the opponent team
merged_df_opponent_wr = pd.merge(filtered_df, lowest_rank_wr_df, left_on='Opponent', right_on='Team', suffixes=('_WR', '_WR_Opponent'))

# Step 13: Merge the filtered WR DataFrame with the DataFrame containing the second lowest-ranked WRs for the opponent team
merged_df_second_lowest = pd.merge(filtered_df, second_lowest_rank_wr_df, left_on='Opponent', right_on='Team', suffixes=('_WR', '_SecondLowest_WR_Opponent'))

# Step 14: Merge the filtered WR DataFrame with the DataFrame containing the third lowest-ranked WRs for the opponent team
merged_df_third_lowest = pd.merge(filtered_df, third_lowest_rank_wr_df, left_on='Opponent', right_on='Team', suffixes=('_WR', '_ThirdLowest_WR_Opponent'))

# Step 15: Replace the opponent team WR name with the corresponding value from the df DataFrame
def get_opponent_team_wr_value(row):
    wr_name = row['Player_WR_Opponent']
    player_name = row['Player_WR']
    wr_value = df.loc[df['Player'] == player_name, wr_name].values[0]
    return wr_value

def get_second_lowest_wr_value(row):
    wr_name = row['Player_SecondLowest_WR_Opponent']
    player_name = row['Player_WR']
    wr_value = df.loc[df['Player'] == player_name, wr_name].values[0]
    return wr_value

def get_third_lowest_wr_value(row):
    wr_name = row['Player_ThirdLowest_WR_Opponent']
    player_name = row['Player_WR']
    wr_value = df.loc[df['Player'] == player_name, wr_name].values[0]
    return wr_value

merged_df_opponent_wr['Lowest_Rank_WR_Opponent'] = merged_df_opponent_wr.apply(get_opponent_team_wr_value, axis=1)
merged_df_second_lowest['Second_Lowest_Rank_WR_Opponent'] = merged_df_second_lowest.apply(get_second_lowest_wr_value, axis=1)
merged_df_third_lowest['Third_Lowest_Rank_WR_Opponent'] = merged_df_third_lowest.apply(get_third_lowest_wr_value, axis=1)

# Step 16: Create the new DataFrame with Player, Lowest_Rank_QB_SameTeam, Lowest_Rank_QB_Opponent, Lowest_Rank_WR_Opponent, Second_Lowest_Rank_WR_Opponent, Third_Lowest_Rank_WR_Opponent, and Lowest_Rank_TE_Opponent columns
wr_te_df = pd.DataFrame({
    'Player': merged_df_same_team['Player_WR/TE'],
    'Lowest_Rank_QB_SameTeam': merged_df_same_team['Lowest_Rank_QB_SameTeam'],
    'Lowest_Rank_QB_Opponent': merged_df_opponent_team['Lowest_Rank_QB_Opponent'],
    'Lowest_Rank_WR_Opponent': merged_df_opponent_wr['Lowest_Rank_WR_Opponent'],
    'Second_Lowest_Rank_WR_Opponent': merged_df_second_lowest['Second_Lowest_Rank_WR_Opponent'],
    'Third_Lowest_Rank_WR_Opponent': merged_df_third_lowest['Third_Lowest_Rank_WR_Opponent'],
    'Lowest_Rank_TE_Opponent': merged_df_opponent_te['Lowest_Rank_TE_Opponent']
})

print(wr_te_df)

               Player  Lowest_Rank_QB_SameTeam  Lowest_Rank_QB_Opponent  \
0    Justin Jefferson                       39                        8   
1      T.J. Hockenson                       29                        9   
2      Jordan Addison                       26                       12   
3         K.J. Osborn                       27                        7   
4       Ja'Marr Chase                       31                       11   
..                ...                      ...                      ...   
209      Trey McBride                       21                        8   
210         Zach Ertz                        6                        2   
211    Michael Wilson                        5                        2   
212       Greg Dortch                        3                        3   
213       Zach Pascal                        0                        0   

     Lowest_Rank_WR_Opponent  Second_Lowest_Rank_WR_Opponent  \
0                          7       

In [144]:
df[['Player', 'Jordan Addison']]

,Player,Jordan Addison
0,Justin Jefferson,15
1,T.J. Hockenson,14
2,Jordan Addison,100
3,Alexander Mattison,8
4,Kirk Cousins,26
...,...,...
350,Zach Ertz,2
351,Michael Wilson,1
352,Greg Dortch,1
353,Keaontay Ingram,1


In [19]:
# Step 1: Filter the DataFrame to keep only players with 'Position' == 'QB'
filtered_df = df[df['Position'] == 'QB'].copy()

# Step 2: Create a DataFrame containing only TEs and their ranks
tes_df = df[df['Position'] == 'TE'][['Team', 'Player', 'Rank']]

# Step 3: Find the lowest-ranked TE for each team
lowest_rank_te_df = tes_df.loc[tes_df.groupby('Team')['Rank'].idxmin()]

# Step 4: Merge the filtered QB DataFrame with the DataFrame containing the lowest-ranked TEs for the opponent team
merged_df = pd.merge(filtered_df, lowest_rank_te_df, left_on='Opponent', right_on='Team', suffixes=('_QB', '_TE'))

# Step 5: Replace the opponent team TE name with the corresponding value from the df DataFrame
def get_te_value(row):
    te_name = row['Player_TE']
    player_name = row['Player_QB']
    te_value = df.loc[df['Player'] == player_name, te_name].values[0]
    return te_value

merged_df['Lowest_Rank_TE_Opponent'] = merged_df.apply(get_te_value, axis=1)

# Step 6: Create a DataFrame containing only WRs and their ranks
wrs_df = df[df['Position'] == 'WR'][['Team', 'Player', 'Rank']]

# Step 7: Find the lowest-ranked WR for each team
lowest_rank_wr_df = wrs_df.loc[wrs_df.groupby('Team')['Rank'].idxmin()]

# Step 8: Find the second lowest-ranked WR for each team
second_lowest_rank_wr_df = wrs_df.loc[wrs_df.groupby('Team')['Rank'].apply(lambda x: x.nsmallest(2).idxmax())]

# Step 9: Find the third lowest-ranked WR for each team
third_lowest_rank_wr_df = wrs_df.loc[wrs_df.groupby('Team')['Rank'].apply(lambda x: x.nsmallest(3).idxmax())]

# Step 10: Merge the filtered QB DataFrame with the DataFrame containing the lowest-ranked WRs for the opponent team
merged_df_wr_opponent = pd.merge(filtered_df, lowest_rank_wr_df, left_on='Opponent', right_on='Team', suffixes=('_QB', '_WR_Opponent'))

# Step 11: Merge the filtered QB DataFrame with the DataFrame containing the second lowest-ranked WRs for the opponent team
merged_df_second_wr_opponent = pd.merge(filtered_df, second_lowest_rank_wr_df, left_on='Opponent', right_on='Team', suffixes=('_QB', '_SecondLowest_WR_Opponent'))

# Step 12: Merge the filtered QB DataFrame with the DataFrame containing the third lowest-ranked WRs for the opponent team
merged_df_third_wr_opponent = pd.merge(filtered_df, third_lowest_rank_wr_df, left_on='Opponent', right_on='Team', suffixes=('_QB', '_ThirdLowest_WR_Opponent'))

# Step 13: Replace the opponent team WR name with the corresponding value from the df DataFrame
def get_wr_value(row):
    wr_name = row['Player_WR_Opponent']
    player_name = row['Player_QB']
    wr_value = df.loc[df['Player'] == player_name, wr_name].values[0]
    return wr_value

def get_second_wr_value(row):
    wr_name = row['Player_SecondLowest_WR_Opponent']
    player_name = row['Player_QB']
    wr_value = df.loc[df['Player'] == player_name, wr_name].values[0]
    return wr_value

def get_third_wr_value(row):
    wr_name = row['Player_ThirdLowest_WR_Opponent']
    player_name = row['Player_QB']
    wr_value = df.loc[df['Player'] == player_name, wr_name].values[0]
    return wr_value

merged_df_wr_opponent['Lowest_Rank_WR_Opponent'] = merged_df_wr_opponent.apply(get_wr_value, axis=1)
merged_df_second_wr_opponent['Second_Lowest_Rank_WR_Opponent'] = merged_df_second_wr_opponent.apply(get_second_wr_value, axis=1)
merged_df_third_wr_opponent['Third_Lowest_Rank_WR_Opponent'] = merged_df_third_wr_opponent.apply(get_third_wr_value, axis=1)

# Step 14: Create a DataFrame containing only WRs and their ranks for the same team
same_team_wr_df = df[df['Position'] == 'WR'][['Team', 'Player', 'Rank']]

# Step 15: Find the lowest-ranked WR for each team in the same team DataFrame
same_team_lowest_rank_wr_df = same_team_wr_df.loc[same_team_wr_df.groupby('Team')['Rank'].idxmin()]

# Step 16: Find the second lowest-ranked WR for each team in the same team DataFrame
same_team_second_lowest_rank_wr_df = same_team_wr_df.loc[same_team_wr_df.groupby('Team')['Rank'].apply(lambda x: x.nsmallest(2).idxmax())]

# Step 17: Find the third lowest-ranked WR for each team in the same team DataFrame
same_team_third_lowest_rank_wr_df = same_team_wr_df.loc[same_team_wr_df.groupby('Team')['Rank'].apply(lambda x: x.nsmallest(3).idxmax())]

# Step 18: Merge the filtered QB DataFrame with the DataFrame containing the lowest-ranked WRs for the same team
merged_df_wr_same_team = pd.merge(filtered_df, same_team_lowest_rank_wr_df, left_on='Team', right_on='Team', suffixes=('_QB', '_WR_SameTeam'))

# Step 19: Merge the filtered QB DataFrame with the DataFrame containing the second lowest-ranked WRs for the same team
merged_df_second_wr_same_team = pd.merge(filtered_df, same_team_second_lowest_rank_wr_df, left_on='Team', right_on='Team', suffixes=('_QB', '_SecondLowest_WR_SameTeam'))

# Step 20: Merge the filtered QB DataFrame with the DataFrame containing the third lowest-ranked WRs for the same team
merged_df_third_wr_same_team = pd.merge(filtered_df, same_team_third_lowest_rank_wr_df, left_on='Team', right_on='Team', suffixes=('_QB', '_ThirdLowest_WR_SameTeam'))

# Step 21: Replace the same team WR name with the corresponding value from the df DataFrame
def get_same_team_wr_value(row):
    wr_name = row['Player_WR_SameTeam']
    player_name = row['Player_QB']
    wr_value = df.loc[df['Player'] == player_name, wr_name].values[0]
    return wr_value

def get_same_team_second_wr_value(row):
    wr_name = row['Player_SecondLowest_WR_SameTeam']
    player_name = row['Player_QB']
    wr_value = df.loc[df['Player'] == player_name, wr_name].values[0]
    return wr_value

def get_same_team_third_wr_value(row):
    wr_name = row['Player_ThirdLowest_WR_SameTeam']
    player_name = row['Player_QB']
    wr_value = df.loc[df['Player'] == player_name, wr_name].values[0]
    return wr_value

merged_df_wr_same_team['Lowest_Rank_WR_SameTeam'] = merged_df_wr_same_team.apply(get_same_team_wr_value, axis=1)
merged_df_second_wr_same_team['Second_Lowest_Rank_WR_SameTeam'] = merged_df_second_wr_same_team.apply(get_same_team_second_wr_value, axis=1)
merged_df_third_wr_same_team['Third_Lowest_Rank_WR_SameTeam'] = merged_df_third_wr_same_team.apply(get_same_team_third_wr_value, axis=1)

# Step 14: Create the new DataFrame with Player, Lowest_Rank_TE_Opponent, Lowest_Rank_WR_Opponent, Second_Lowest_Rank_WR_Opponent, Third_Lowest_Rank_WR_Opponent, and Same_Team_TE columns
same_team_te_df = pd.merge(filtered_df, lowest_rank_te_df, left_on='Team', right_on='Team', suffixes=('_QB', '_TE_SameTeam'))

def get_te_value_same_team(row):
    te_name = row['Player_TE_SameTeam']
    player_name = row['Player_QB']
    te_value = df.loc[df['Player'] == player_name, te_name].values[0]
    return te_value

same_team_te_df['Lowest_Rank_TE_SameTeam'] = same_team_te_df.apply(get_te_value_same_team, axis=1)

# Step 22: Create the new DataFrame with Player, Lowest_Rank_TE_Opponent, Lowest_Rank_WR_Opponent, Second_Lowest_Rank_WR_Opponent, Third_Lowest_Rank_WR_Opponent, Same_Team_TE, Same_Team_WR, Same_Team_Second_WR, and Same_Team_Third_WR columns
qb_df = pd.DataFrame({
    'Player': merged_df['Player_QB'],
    'Lowest_Rank_TE': merged_df['Lowest_Rank_TE_Opponent'],
    'Lowest_Rank_WR': merged_df_wr_opponent['Lowest_Rank_WR_Opponent'],
    'Second_Lowest_Rank_WR': merged_df_second_wr_opponent['Second_Lowest_Rank_WR_Opponent'],
    'Third_Lowest_Rank_WR': merged_df_third_wr_opponent['Third_Lowest_Rank_WR_Opponent'],
    'Same_Team_TE': same_team_te_df['Lowest_Rank_TE_SameTeam'],
    'Same_Team_WR': merged_df_wr_same_team['Lowest_Rank_WR_SameTeam'],
    'Same_Team_Second_WR': merged_df_second_wr_same_team['Second_Lowest_Rank_WR_SameTeam'],
    'Same_Team_Third_WR': merged_df_third_wr_same_team['Third_Lowest_Rank_WR_SameTeam']
})

print(qb_df)


                Player  Lowest_Rank_TE  Lowest_Rank_WR  Second_Lowest_Rank_WR  \
0         Kirk Cousins               6               8                     12   
1           Joe Burrow               5              10                     15   
2           Trey Lance               0               4                      6   
3          Brock Purdy               2               9                      9   
4          Sam Darnold               0               1                      0   
5       Tua Tagovailoa              13               8                      6   
6           Mike White               0               1                      0   
7     Matthew Stafford              11               7                      6   
8      Patrick Mahomes              10              11                      5   
9           Josh Allen              12              14                     10   
10         Jalen Hurts               8               7                     11   
11      Marcus Mariota      